# Levenberg Marquardt Fitting


## Conventions

-   S0 Signal for unbound state
-   S1 Signal for bound state
-   K equilibrium constant (Kd or pKa)
-   order data from unbound to bound (e.g. cl: 0–\>150 mM; pH 9–\>5)


In [ ]:
import arviz as az
import corner
import lmfit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import scipy
import seaborn as sb

from clophfit.binding.fitting import Dataset, fit_binding_glob
from clophfit.binding.plotting import plot_emcee, print_emcee

%load_ext autoreload
%autoreload 2

## Single Cl titration.

In [ ]:
df = pd.read_table("../../tests/data/copyIP.txt")
df["F"] /= np.max(df["F"])
sb.scatterplot(data=df, x="cl", y="F", hue=df.cl, palette="crest", s=200)

In general we can use either `lmfit.minimize() -> res` or `lmfit.Minimizer -> mini`.

In [ ]:
def fz(x, S0, S1, Kd):
    return (S0 + S1 * x / Kd) / (1 + x / Kd)


def residual(pars, x, y=None):
    S0 = pars["S0"]
    S1 = pars["S1"]
    Kd = pars["Kd"]
    model = fz(x, S0, S1, Kd)
    if y is None:
        return model
    return y - model


params = lmfit.Parameters()
params.add("S0", value=df.F[0], min=0, max=2)
params.add("S1", value=100, min=-0.1, max=2)
params.add("Kd", value=50, min=0, max=200)
res = lmfit.minimize(residual, params, args=(df.cl, df.F))

xdelta = (df.cl.max() - df.cl.min()) / 500
xfit = np.arange(df.cl.min() - xdelta, df.cl.max() + xdelta, xdelta)
yfit = residual(res.params, xfit)
print(lmfit.fit_report(res.params))
plt.plot(df.cl, df.F, "o", xfit, yfit, "-")

In [ ]:
mini = lmfit.Minimizer(residual, params, fcn_args=(df.cl, df.F))
res = mini.minimize()
ci, tr = lmfit.conf_interval(mini, res, sigmas=[0.68, 0.95], trace=True)
print(lmfit.ci_report(ci, with_offset=False, ndigits=2))
print(lmfit.fit_report(res, show_correl=False, sort_pars=True))

In [ ]:
mini.params.add("__lnsigma", value=np.log(0.1), min=np.log(0.001), max=np.log(2))
emcee_res = mini.emcee(
    nwalkers=40,
    workers=8,
    is_weighted=False,
    thin=80,
    burn=300,
    steps=4000,
    progress=False,
)

In [ ]:
fig = plt.figure(figsize=(7, 7))
f = corner.corner(emcee_res.flatchain.to_numpy(), fig=fig)

In [ ]:
# Extract parameter values and standard errors from lmfit result
S0_mu = res.params["S0"].value
S1_mu = res.params["S1"].value
Kd_mu = res.params["Kd"].value
S0_std = res.params["S0"].stderr
S1_std = res.params["S1"].stderr
Kd_std = res.params["Kd"].stderr

cov_matrix = res.covar

# Define PyMC3 model
with pm.Model() as model:
    # Define multivariate normal priors for parameters
    pars = pm.MvNormal("pars", mu=[S0_mu, S1_mu, Kd_mu], cov=cov_matrix, shape=3)
    S0 = pars[0]
    S1 = pars[1]
    Kd = pars[2]

    y_pred = pm.Deterministic("y_pred", fz(df.cl.to_numpy(), S0, S1, Kd))
    # Define likelihood
    likelihood = pm.Normal("likelihood", mu=y_pred, sigma=1, observed=df.F.to_numpy())
    # Run the inference
    trace = pm.sample(2000, tune=1000, cores=8, progressbar=False)

In [ ]:
az.plot_posterior(
    trace,
    var_names=["pars"],
    hdi_prob=0.95,
    textsize=11,
    round_to=3,
    point_estimate="median",
    ref_val=[res.params["S0"].value, res.params["S1"].value, res.params["Kd"].value],
    figsize=(12, 3),
)

In [ ]:
pm.summary(trace)

In [ ]:
f = corner.corner(trace)

In [ ]:
pm.plot_trace(trace, combined=True, rug=True)

In [ ]:
def plot_scatter_matrix(res, tr):
    names = res.params.keys()
    gs = plt.GridSpec(4, 4)
    sx = {}
    sy = {}

    for i, fixed in enumerate(names):
        for j, free in enumerate(names):
            sharex = sx.get(j)
            sharey = sy.get(i)
            ax = plt.subplot(gs[i, j], sharex=sharex, sharey=sharey)

            if sharex is None:
                sx[j] = ax
            if sharey is None:
                sy[i] = ax

            if i < 3:
                plt.setp(ax.get_xticklabels(), visible=True)
            else:
                ax.set_xlabel(free)

            if j > 0:
                plt.setp(ax.get_yticklabels(), visible=False)
            else:
                ax.set_ylabel(fixed)

            rest = tr[fixed]
            prob = rest["prob"]
            f = prob < 0.96

            x, y = rest[free], rest[fixed]
            ax.scatter(x[f], y[f], c=1 - prob[f], s=25 * (1 - prob[f] + 0.5))
            ax.autoscale(1, 1)


plot_scatter_matrix(res, tr)

The plots shown below, akin to the examples provided in [lmfit documentation](https://lmfit.github.io/lmfit-py/examples/example_confidence_interval.html#sphx-glr-examples-example-confidence-interval-py), are computationally intensive. They operate under the assumption of a parabolic parameter space. However, it's worth noting that these plots provide similar information to that yielded by a Monte Carlo simulation.

In [ ]:
def plot_conf_interval_matrix(res, tr):
    names = list(res.params.keys())
    plt.figure()

    for i in range(3):
        for j in range(3):
            indx = 9 - j * 3 - i
            ax = plt.subplot(3, 3, indx)
            ax.ticklabel_format(style="sci", scilimits=(-2, 2), axis="y")

            # Set-up labels and tick marks
            ax.tick_params(labelleft=False, labelbottom=False)
            if indx in (1, 4, 7):
                plt.ylabel(names[j])
                ax.tick_params(labelleft=True)
            if indx == 1:
                ax.tick_params(labelleft=True)
            if indx in (7, 8, 9):
                plt.xlabel(names[i])
                ax.tick_params(labelbottom=True)
                [label.set_rotation(45) for label in ax.get_xticklabels()]

            if i != j:
                x, y, m = lmfit.conf_interval2d(mini, res, names[i], names[j], 20, 20)
                plt.contourf(x, y, m, np.linspace(0, 1, 10))

                x = tr[names[i]][names[i]]
                y = tr[names[i]][names[j]]
                pr = tr[names[i]]["prob"]
                s = np.argsort(x)
                plt.scatter(x[s], y[s], c=pr[s], s=30, lw=1)

            else:
                x = tr[names[i]][names[i]]
                y = tr[names[i]]["prob"]

                t, s = np.unique(x, True)
                f = scipy.interpolate.interp1d(t, y[s], "slinear")
                xn = np.linspace(x.min(), x.max(), 50)
                plt.plot(xn, f(xn), lw=1)
                plt.ylabel("prob")
                ax.tick_params(labelleft=True)

    plt.tight_layout()


plot_conf_interval_matrix(res, tr)

In [ ]:
def plot_conf_intervals(mini, res):
    # Report fit parameters
    lmfit.report_fit(res.params, min_correl=0.25)

    # Calculate confidence intervals and traces
    ci, trace = lmfit.conf_interval(mini, res, sigmas=[1, 2], trace=True)
    lmfit.printfuncs.report_ci(ci)

    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(12.8, 9.6), sharey=True)

    # Plot scatter plots for S0 and S1
    plot_scatter_trace(axes[0][0], trace["S0"], "S0", "Kd")
    plot_scatter_trace(axes[0][1], trace["S1"], "S1", "Kd")

    # Plot 2D confidence intervals for S0 and S1
    plot_2d_conf_interval(axes[1][0], mini, res, "S0", "Kd")
    plot_2d_conf_interval(axes[1][1], mini, res, "S1", "Kd")

    plt.tight_layout()


def plot_scatter_trace(ax, trace_data, xlabel, ylabel):
    cx, cy, prob = trace_data[xlabel], trace_data[ylabel], trace_data["prob"]
    ax.scatter(cx, cy, c=prob, s=30)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)


def plot_2d_conf_interval(ax, mini, res, xparam, yparam):
    # Calculate 2D confidence interval
    cx, cy, grid = lmfit.conf_interval2d(mini, res, xparam, yparam, 30, 30)
    # Plot the contourf with a colorbar
    ctp = ax.contourf(cx, cy, grid, np.linspace(0, 1, 11))
    fig = ax.figure
    fig.colorbar(ctp, ax=ax)
    ax.set_xlabel(xparam)
    ax.set_ylabel(yparam)


plot_conf_intervals(mini, res)

In [ ]:
f, ax = plt.subplots(1, 1)
plot_2d_conf_interval(ax, mini, res, "S0", "S1")

### Using clophfit.binding

In [ ]:
ds = Dataset(df["cl"].to_numpy(), df["F"].to_numpy())
f_res = fit_binding_glob(ds, weighting=True)

f_res.figure

In [ ]:
emcee_res = f_res.mini.emcee(
    steps=3000, burn=300, workers=8, nwalkers=30, seed=1, progress=False
)
plot_emcee(emcee_res.flatchain)
print_emcee(emcee_res)

In [ ]:
emcee_res.flatchain.describe()

## Fit titration with multiple datasets

For example data collected with multiple labelblocks in Tecan plate reader.

“A01”: pH titration with y1 and y2.

In [ ]:
df = pd.read_csv("../../tests/data/A01.dat", sep=" ", names=["x", "y1", "y2"])
df = df[::-1].reset_index(drop=True)
df

### lmfit of single y1 using analytical Jacobian

It computes the Jacobian of the fz. Mind that the residual (i.e. y - fz)
will be actually minimized.

In [ ]:
import sympy

x, S0_1, S1_1, K = sympy.symbols("x S0_1 S1_1 K")
f = (S0_1 + S1_1 * 10 ** (K - x)) / (1 + 10 ** (K - x))
print(sympy.diff(f, S0_1))
print(sympy.diff(f, S1_1))
print(sympy.diff(f, K))

In [ ]:
x, S0, S1, K = sympy.symbols("x S0 S1 K")
f = S0 + (S1 - S0) * x / K / (1 + x / K)
sympy.diff(f, S0)

In [ ]:
sympy.diff(f, S1)

In [ ]:
sympy.diff(f, K)

In [ ]:
# if is_ph:
f = S0 + (S1 - S0) * 10 ** (K - x) / (1 + 10 ** (K - x))
sympy.diff(f, S0)

In [ ]:
sympy.diff(f, S1)

In [ ]:
sympy.diff(f, K)

In [ ]:
def residual(pars, x, data):
    S0 = pars["S0"]
    S1 = pars["S1"]
    K = pars["K"]
    # model = (S0 + S1 * x / Kd) / (1 + x / Kd)
    x = np.array(x)
    y = np.array(data)
    model = (S0 + S1 * 10 ** (K - x)) / (1 + 10 ** (K - x))
    if data is None:
        return model
    return y - model


# Try Jacobian
def dfunc(pars, x, data=None):
    if data is None:
        pass
    S0_1 = pars["S0"]
    S1_1 = pars["S1"]
    K = pars["K"]
    kx = np.array(10 ** (K - x))
    return np.array(
        [
            -1 / (kx + 1),
            -kx / (kx + 1),
            -kx * np.log(10) * (S1_1 / (kx + 1) - (kx * S1_1 + S0_1) / (kx + 1) ** 2),
        ]
    )


params = lmfit.Parameters()
params.add("S0", value=25000)
params.add("S1", value=50000, min=0.0)
params.add("K", value=7, min=2.0, max=12.0)

mini = lmfit.Minimizer(residual, params, fcn_args=(df.x,), fcn_kws={"data": df.y1})
res = mini.leastsq(Dfun=dfunc, col_deriv=True, ftol=1e-8)
print(lmfit.report_fit(res))
ci = lmfit.conf_interval(mini, res, sigmas=[1, 2, 3])

In [ ]:
print(lmfit.ci_report(ci, with_offset=False, ndigits=2))

### using lmfit with np.r_ trick

In [ ]:
def residual2(pars, x, data=None):
    K = pars["K"]
    S0_1 = pars["S0_1"]
    S1_1 = pars["S1_1"]
    S0_2 = pars["S0_2"]
    S1_2 = pars["S1_2"]
    model_0 = (S0_1 + S1_1 * 10 ** (K - x[0])) / (1 + 10 ** (K - x[0]))
    model_1 = (S0_2 + S1_2 * 10 ** (K - x[1])) / (1 + 10 ** (K - x[1]))
    if data is None:
        return np.r_[model_0, model_1]
    return np.r_[data[0] - model_0, data[1] - model_1]


params2 = lmfit.Parameters()
params2.add("K", value=7.0, min=2.0, max=12.0)
params2.add("S0_1", value=df.y1[0], min=0.0)
params2.add("S0_2", value=df.y2[0], min=0.0)
params2.add("S1_1", value=df.y1.iloc[-1], min=0.0)
params2.add("S1_2", value=df.y2.iloc[-1], min=0.0)
mini2 = lmfit.Minimizer(
    residual2, params2, fcn_args=([df.x, df.x],), fcn_kws={"data": [df.y1, df.y2]}
)
res2 = mini2.minimize()
print(lmfit.fit_report(res2))

ci2, tr2 = lmfit.conf_interval(mini2, res2, sigmas=[0.68, 0.95], trace=True)
print(lmfit.ci_report(ci2, with_offset=False, ndigits=2))

In [ ]:
xfit = np.linspace(df.x.min(), df.x.max(), 100)
yfit0 = residual2(params2, [xfit, xfit])
yfit0 = yfit0.reshape(2, 100)
yfit = residual2(res2.params, [xfit, xfit])
yfit = yfit.reshape(2, 100)
plt.plot(df.x, df.y1, "o", df.x, df.y2, "s")
plt.plot(xfit, yfit[0], "-", xfit, yfit[1], "-")
plt.plot(xfit, yfit0[0], "--", xfit, yfit0[1], "--")
plt.grid(True)

### lmfit constraints aiming for generality

I believe a name convention would be more robust than relying on
OrderedDict Params object.

In [ ]:
"S0_1".split("_")[0]

In [ ]:
def exception_fcn_handler(func):
    def inner_function(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except TypeError:
            print(
                f"{func.__name__} only takes (1D) vector as argument besides lmfit.Parameters."
            )

    return inner_function


@exception_fcn_handler
def titration_pH(params, pH):
    p = {k.split("_")[0]: v for k, v in params.items()}
    return (p["S0"] + p["S1"] * 10 ** (p["K"] - pH)) / (1 + 10 ** (p["K"] - pH))


def residues(params, x, y, fcn):
    return y - fcn(params, x)


p1 = lmfit.Parameters()
p2 = lmfit.Parameters()
p1.add("K_1", value=7.0, min=2.0, max=12.0)
p2.add("K_2", value=7.0, min=2.0, max=12.0)
p1.add("S0_1", value=df.y1.iloc[0], min=0.0)
p2.add("S0_2", value=df.y2.iloc[0], min=0.0)
p1.add("S1_1", value=df.y1.iloc[-1], min=0.0)
p2.add("S1_2", value=df.y2.iloc[-1])

print(
    residues(p1, np.array(df.x), [1.97, 1.8, 1.7, 0.1, 0.1, 0.16, 0.01], titration_pH)
)


def gobjective(params, xl, yl, fcnl):
    nset = len(xl)
    res = []
    for i in range(nset):
        pi = {k: v for k, v in params.valuesdict().items() if k[-1] == f"{i+1}"}
        res = np.r_[res, residues(pi, xl[i], yl[i], fcnl[i])]
        # res = np.r_[res, yl[i] - fcnl[i](parsl[i], x[i])]
    return res


print(gobjective(p1 + p2, [df.x, df.x], [df.y1, df.y2], [titration_pH, titration_pH]))

Here single.

In [ ]:
mini = lmfit.Minimizer(
    residues,
    p1,
    fcn_args=(
        df.x,
        df.y1,
        titration_pH,
    ),
)
res = mini.minimize()

fit = titration_pH(res.params, df.x)
print(lmfit.report_fit(res))
plt.plot(df.x, df.y1, "o", df.x, fit, "--")
ci = lmfit.conf_interval(mini, res, sigmas=[1, 2])
lmfit.printfuncs.report_ci(ci)

Now global.

In [ ]:
pg = p1 + p2
pg["K_2"].expr = "K_1"
gmini = lmfit.Minimizer(
    gobjective,
    pg,
    fcn_args=([df.x[:], df.x], [df.y1[:], df.y2], [titration_pH, titration_pH]),
)
gres = gmini.minimize()
print(lmfit.fit_report(gres))

pp1 = {k: v for k, v in gres.params.valuesdict().items() if k.split("_")[1] == f"{1}"}
pp2 = {k: v for k, v in gres.params.valuesdict().items() if k.split("_")[1] == f"{2}"}
xfit = np.linspace(df.x.min(), df.x.max(), 100)
yfit1 = titration_pH(pp1, xfit)
yfit2 = titration_pH(pp2, xfit)
plt.plot(df.x, df.y1, "o", xfit, yfit1, "--")
plt.plot(df.x, df.y2, "s", xfit, yfit2, "--")

In [ ]:
ci = lmfit.conf_interval(gmini, gres)
print(lmfit.ci_report(ci, with_offset=False, ndigits=2))

To plot ci for the 5 parameters.

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(24.2, 4.8), sharey=True)
cx, cy, grid = lmfit.conf_interval2d(gmini, gres, "S0_1", "K_1", 25, 25)
ctp = axes[0].contourf(cx, cy, grid, np.linspace(0, 1, 11))
fig.colorbar(ctp, ax=axes[0])
axes[0].set_xlabel("SA1")
axes[0].set_ylabel("pK1")
cx, cy, grid = lmfit.conf_interval2d(gmini, gres, "S0_2", "K_1", 25, 25)
ctp = axes[1].contourf(cx, cy, grid, np.linspace(0, 1, 11))
fig.colorbar(ctp, ax=axes[1])
axes[1].set_xlabel("SA2")
axes[1].set_ylabel("pK1")
cx, cy, grid = lmfit.conf_interval2d(gmini, gres, "S1_1", "K_1", 25, 25)
ctp = axes[2].contourf(cx, cy, grid, np.linspace(0, 1, 11))
fig.colorbar(ctp, ax=axes[2])
axes[2].set_xlabel("SB1")
axes[2].set_ylabel("pK1")
cx, cy, grid = lmfit.conf_interval2d(gmini, gres, "S1_2", "K_1", 25, 25)
ctp = axes[3].contourf(cx, cy, grid, np.linspace(0, 1, 11))
fig.colorbar(ctp, ax=axes[3])
axes[3].set_xlabel("SB2")
axes[3].set_ylabel("pK1")

In [ ]:
plt.plot(np.r_[df.x, df.x], gres.residual, "o")
std = gres.residual.std()
for i in range(-3, 4):
    plt.hlines(i * std, 5, 9, alpha=0.4)
print(std)

This next block comes from:
<https://lmfit.github.io/lmfit-py/examples/example_emcee_Model_interface.html?highlight=emcee>

### lmfit.Model

It took 9 vs 5 ms. It is not possible to do global fitting. In the
documentation it is stressed the need to convert the output of the
residue to be 1D vectors.

In [ ]:
mod = lmfit.models.ExpressionModel("(SB + SA * 10**(pK-x)) / (1 + 10**(pK-x))")
result = mod.fit(np.array(df.y1), x=np.array(df.x), pK=7, SB=7e3, SA=10000)
print(result.fit_report())

In [ ]:
plt.plot(df.x, df.y1, "o")
plt.plot(df.x, result.init_fit, "--", label="initial fit")
plt.plot(df.x, result.best_fit, "-", label="best fit")
plt.legend()

In [ ]:
print(result.ci_report())

which is faster but still I failed to find the way to global fitting.

In [ ]:
def tit_pH(x, S0, S1, K):
    return (S0 + S1 * 10 ** (K - x)) / (1 + 10 ** (K - x))


tit_model1 = lmfit.Model(tit_pH, prefix="ds1_")
tit_model2 = lmfit.Model(tit_pH, prefix="ds2_")
print(f"parameter names: {tit_model1.param_names}")
print(f"parameter names: {tit_model2.param_names}")
print(f"independent variables: {tit_model1.independent_vars}")
print(f"independent variables: {tit_model2.independent_vars}")

tit_model1.set_param_hint("K", value=7.0, min=2.0, max=12.0)
tit_model1.set_param_hint("S0", value=df.y1[0], min=0.0)
tit_model1.set_param_hint("S1", value=df.y1.iloc[-1], min=0.0)
tit_model2.set_param_hint("K", value=7.0, min=2.0, max=12.0)
tit_model2.set_param_hint("S0", value=df.y1[0], min=0.0)
tit_model2.set_param_hint("S1", value=df.y1.iloc[-1], min=0.0)
pars1 = tit_model1.make_params()
pars2 = tit_model2.make_params()
# gmodel = tit_model1 + tit_model2
# result = gmodel.fit(df.y1 + df.y2, pars, x=df.x)
res1 = tit_model1.fit(df.y1, pars1, x=df.x)
res2 = tit_model2.fit(df.y2, pars2, x=df.x)
print(res1.fit_report())
print(res2.fit_report())

In [ ]:
xfit_delta = (df.x.max() - df.x.min()) / 100
xfit = np.arange(df.x.min() - xfit_delta, df.x.max() + xfit_delta, xfit_delta)
dely1 = res1.eval_uncertainty(x=xfit) * 1
dely2 = res2.eval_uncertainty(x=xfit) * 1
best_fit1 = res1.eval(x=xfit)
best_fit2 = res2.eval(x=xfit)
plt.plot(df.x, df.y1, "o")
plt.plot(df.x, df.y2, "o")
plt.plot(xfit, best_fit1, "-.")
plt.plot(xfit, best_fit2, "-.")
plt.fill_between(xfit, best_fit1 - dely1, best_fit1 + dely1, color="#FEDCBA", alpha=0.5)
plt.fill_between(xfit, best_fit2 - dely2, best_fit2 + dely2, color="#FEDCBA", alpha=0.5)

Please mind the difference in the uncertainty between the 2 label
blocks.

In [ ]:
def tit_pH2(x, S0_1, S0_2, S1_1, S1_2, K):
    y1 = (S0_1 + S1_1 * 10 ** (K - x)) / (1 + 10 ** (K - x))
    y2 = (S0_2 + S1_2 * 10 ** (K - x)) / (1 + 10 ** (K - x))
    # return y1, y2
    return np.r_[y1, y2]


tit_model = lmfit.Model(tit_pH2)
tit_model.set_param_hint("K", value=7.0, min=2.0, max=12.0)
tit_model.set_param_hint("S0_1", value=df.y1[0], min=0.0)
tit_model.set_param_hint("S0_2", value=df.y2[0], min=0.0)
tit_model.set_param_hint("S1_1", value=df.y1.iloc[-1], min=0.0)
tit_model.set_param_hint("S1_2", value=df.y2.iloc[-1], min=0.0)
pars = tit_model.make_params()
# res = tit_model.fit([df.y1, df.y2], pars, x=df.x)
res = tit_model.fit(np.r_[df.y1, df.y2], pars, x=df.x)
print(res.fit_report())

In [ ]:
dely = res.eval_uncertainty(x=xfit)

In [ ]:
def fit_pH(fp):
    df = pd.read_csv(fp)

    def tit_pH(x, SA, SB, pK):
        return (SB + SA * 10 ** (pK - x)) / (1 + 10 ** (pK - x))

    mod = lmfit.Model(tit_pH)
    pars = mod.make_params(SA=10000, SB=7e3, pK=7)
    result = mod.fit(df.y2, pars, x=df.x)
    return result, df.y2, df.x, mod


# r,y,x,model = fit_pH("/home/dati/ibf/IBF/Database/Random mutag results/Liasan-analyses/2016-05-19/2014-02-20/pH/dat/C12.dat")
r, y, x, model = fit_pH("../../tests/data/H04.dat")
xfit = np.linspace(x.min(), x.max(), 50)
dely = r.eval_uncertainty(x=xfit) * 1
best_fit = r.eval(x=xfit)
plt.plot(x, y, "o")
plt.plot(xfit, best_fit, "-.")
plt.fill_between(xfit, best_fit - dely, best_fit + dely, color="#FEDCBA", alpha=0.5)
r.conf_interval(sigmas=[2])
print(r.ci_report(with_offset=False, ndigits=2))

In [ ]:
g = r.plot()

In [ ]:
print(r.ci_report())

### using clophfit.binding

In [ ]:
dictionary = df.loc[:, ["y1", "y2"]].to_dict(orient="series")
dictionary = {key: value.to_numpy() for key, value in dictionary.items()}
ds = Dataset(df.x.to_numpy(), dictionary, True)
f_res2 = fit_binding_glob(ds, True)
f_res2.figure

In [ ]:
emcee_res2 = f_res2.mini.emcee(
    steps=4000, burn=100, workers=8, nwalkers=40, seed=1, progress=False
)
plot_emcee(emcee_res2.flatchain)
print_emcee(emcee_res2)

## Model: example 2P Cl–ratio

In [ ]:
filepath = "../../tests/data/ratio2P.txt"
df = pd.read_table(filepath)


def R_Cl(cl, R0, R1, K):
    return (R1 * cl + R0 * K) / (K + cl)


def fit_Rcl(df):
    mod = lmfit.Model(R_Cl)
    # pars = mod.make_params(R0=0.8, R1=0.05, K=10)
    pars = lmfit.Parameters()
    pars.add("R0", value=df.R[0], min=0.2, max=1.2)
    pars.add("R1", value=0.05, min=-0.4, max=0.6)
    pars.add("K", value=10, min=0, max=60)
    result = mod.fit(df.R, pars, cl=df.cl)
    return result, mod


def plot_fit(result, x, y):
    """Plot the original data and the best fit line with uncertainty."""
    xfit = np.linspace(x.min(), x.max(), 50)
    dely = result.eval_uncertainty(cl=xfit) * 3
    best_fit = result.eval(cl=xfit)
    plt.plot(x, y, "o")
    plt.grid()
    plt.plot(xfit, best_fit, "-.")
    plt.fill_between(xfit, best_fit - dely, best_fit + dely, color="#FEDCBA", alpha=0.5)
    result.conf_interval(sigmas=[2])
    print(result.ci_report(with_offset=False, ndigits=2))


result, model = fit_Rcl(df)
plot_fit(result, df.cl, df.R)

In [ ]:
emcee_kws = {
    "is_weighted": False,
    "steps": 2000,
    "burn": 150,
    "thin": 20,
    "nwalkers": 60,
    "seed": 11,
    "workers": 8,
    "progress": False,
}
emcee_params = result.params.copy()
emcee_params.add("__lnsigma", value=np.log(0.1), min=np.log(0.001), max=np.log(2.0))

result_emcee = model.fit(
    data=df.R,
    cl=df.cl,
    params=emcee_params,
    method="emcee",
    scale_covar=1,
    nan_policy="omit",
    fit_kws=emcee_kws,
)

In [ ]:
lmfit.report_fit(result_emcee)

In [ ]:
f = plot_emcee(result_emcee.flatchain)

In [ ]:
print_emcee(result_emcee)

In [ ]:
def residual(pars, x, data=None):
    """Model a decaying sine wave and subtract data."""
    vals = pars.valuesdict()
    R0 = vals["R0"]
    R1 = vals["R1"]
    K = vals["K"]
    model = R_Cl(x, R0, R1, K)
    if data is None:
        return model
    return model - data


params = lmfit.Parameters()
params.add("R0", value=df["R"][0], min=0, max=1)
params.add("R1", value=df["R"].iloc[-1], min=0, max=0.2)
target_y = (df["R"][0] + df["R"].iloc[-1]) / 2
k_initial = df["cl"][np.argmin(np.abs(df["R"] - target_y))]
params.add("K", value=k_initial, min=3, max=30)
mini = lmfit.Minimizer(residual, params, fcn_args=(df["cl"], df["R"]))
result = mini.minimize()
# Print a report of the fit
lmfit.report_fit(result)

In [ ]:
emcee_params.add("__lnsigma", value=np.log(0.1), min=np.log(0.01), max=np.log(0.1))

emcee_res3 = mini.emcee(
    steps=4000,
    burn=300,
    workers=16,
    nwalkers=30,
    seed=1,
    is_weighted=False,
    progress=False,
)
plot_emcee(emcee_res3.flatchain)
print_emcee(emcee_res3)
# out = lmfit.minimize(residual, params, args=(df["cl"],), kws={'data': df["R"]})

In [ ]:
import pymc as pm

with pm.Model() as model:
    R0 = pm.Normal("R0", mu=result.params["R0"].value, sigma=result.params["R0"].stderr)
    R1 = pm.Normal("R1", mu=result.params["R1"].value, sigma=result.params["R1"].stderr)
    K = pm.Normal("K", mu=result.params["K"].value, sigma=result.params["K"].stderr)

    y_pred = pm.Deterministic("y_pred", R_Cl(df["cl"], R0, R1, K))

    likelihood = pm.Normal("y", mu=y_pred, sigma=1, observed=df["R"])

    trace = pm.sample(2000, tune=2000, chains=6, cores=8)

In [ ]:
R0_samples = trace.posterior["R0"]
az.hdi(R0_samples, hdi_prob=0.95)["R0"]

In [ ]:
# Extract the posterior samples for the parameters of interest
K_samples = trace.posterior["K"].to_numpy().flatten()
R0_samples = trace.posterior["R0"].to_numpy().flatten()
R1_samples = trace.posterior["R1"].to_numpy().flatten()
# Ensure the samples are in the correct format for the corner plot
samples_array = np.column_stack([K_samples, R0_samples, R1_samples])
# Plot the corner plot
f = corner.corner(samples_array, labels=["K", "R0", "R1"])

In [ ]:
ax = az.plot_pair(
    trace,
    divergences=1,
    var_names=["K", "R0", "R1"],
    kind=["kde", "scatter"],
    kde_kwargs={"fill_last": False},
    marginals=True,
    point_estimate="mean",
    figsize=(9, 9),
)

In [ ]:
pm.plot_trace(trace)

In [ ]:
pm.plot_posterior(
    trace,
    var_names=["R0", "R1", "K"],
    figsize=(12, 4),
    textsize=12,
)